In [1]:
import json
import numpy as np
from pprint import pprint
from os import listdir
from os.path import isfile, join
from MongoOps import MongoOps
from ICSParser import ICSParser
from make_directions_matrix import DirectionsMatrix

In [3]:
locations = []
mops = MongoOps()
sample_data_files = [f for f in listdir("/data") if isfile(join("/data", f))]
for open_house_file in sample_data_files:
    parser = ICSParser("/data/%s" % open_house_file)
    event = parser.to_dict()
    # print(event)
    result = mops.safe_query_for_location_info(event)
    locations += [result]

In [4]:
DM = DirectionsMatrix(locations, mops)

In [5]:
DM.generate_simplified_directions_matrix()

In [6]:
sdm = DM.simplified_directions_matrix

In [7]:
# Showing off EST/EDT time of day of the open houses and the conversion to minutes from midnight that day
for i in range(len(sdm)):
    start = int(sdm[i]['dtstart'][9:-3])-600
    start_minutes = int(60/100*start)
    end = int(sdm[i]['dtend'][9:-3])-600
    end_minutes = int(60/100*end)
    print('%d (%d), %d (%d)' % (start, start_minutes, end, end_minutes))
    start = sdm[i]['start'] = start
    start_minutes = sdm[i]['start_minutes'] = start_minutes
    end = sdm[i]['end'] = end
    end_minutes = sdm[i]['end_minutes'] = end_minutes

1100 (660), 1400 (840)
900 (540), 1100 (660)
1000 (600), 1200 (720)
1200 (720), 1400 (840)
1000 (600), 1130 (678)
1000 (600), 1200 (720)
1000 (600), 1200 (720)
1000 (600), 1200 (720)
1100 (660), 1300 (780)
1200 (720), 1400 (840)
1100 (660), 1300 (780)


In [8]:
vertices = []
V = None
for i in range(len(sdm)):
    V = {'ID' : i, 
               'start' : sdm[i]['start_minutes'], 
               'end' : sdm[i]['end_minutes'], 
               'edges' : sdm[i]['durations'],
               'address_hash' : sdm[i]['address_hash']}
    vertices += [V]

In [9]:
def been_visited(visited, v):
    '''
    Checks to make sure you're not going back to a node that has already been visited.
    '''
    return v in visited

In [10]:
""" 
A Python Class
A simple Python graph class, demonstrating the essential 
facts and functionalities of graphs.
Original implementation from https://www.python-course.eu/graphs_python.php
Changes to include weighted edges from https://towardsdatascience.com/to-all-data-scientists-the-one-graph-algorithm-you-need-to-know-59178dbb1ec2
Some functions have been removed because they are not 
going to be used, and I would like to protect future 
users from using this graph object incorrectly.
"""

class Graph(object):
    def __init__(self, graph_dict=None):
        """ initializes a graph object 
            If no dictionary or None is given, 
            an empty dictionary will be used
        """
        if graph_dict == None:
            graph_dict = {}
        self.__graph_dict = graph_dict
        self.__vertices = self.vertices()

    def vertex_ids(self):
        """ returns the vertices of a graph """
        vertices = []
        for val in self.__graph_dict:
            vertices += [val['ID']]
        self.__vertices = vertices
        return list(self.__vertices)
    
    def vertices(self):
        """ returns the vertices of a graph """
        vertices = []
        for val in self.__graph_dict:
            ver = self.__removekey(val, 'edges')
            vertices += [ver]
        self.__vertices = vertices
        return list(self.__vertices)
    
    def get_vertex_from_vid(self, vid):
        ''' 
        returns the vertex given its id.
        I would LIKE to assume that the vertex's id will match its location in
        graph_dict, but I won't in the case that someone passes in an irregualar
        graph_dict to the Graph object.
        '''
        vertex = None
        for val in self.__graph_dict:
            if val['ID'] == vid:
                vertex = val
        if vertex == None:
            raise Exception('Vertex ID provided not found.')
        return vertex

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()
    
    def get_edges_from_vid(self, vid):
        ''' 
        returns the edges from a vertex, given its id.
        '''
        vertex = self.get_vertex_from_vid(vid)
        edges = self.get_edges(vertex)
        return edges

    def __generate_edges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self.__graph_dict:
            edges += self.get_edges(vertex)
        return edges

    def get_edges(self, vertex):
        edges = []
        for neighbour in vertex['edges']:
            weight = neighbour[1]
            neighbour = neighbour[0]
            v = vertex['ID']
            if (neighbour, vertex, weight) not in edges:
                edges.append([v, neighbour, weight])
        return edges
    
    def __str__(self):
        res = "vertices:\n"
        for k in self.__vertices:
            res += str(k) + " \n"
        res += "\nedges:\n"
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res

    def adj_mat(self):
        return self.__graph_dict
    
    def __removekey(self, d, key):
        r = dict(d)
        del r[key]
        return r

In [11]:
graph = Graph(vertices)
print(graph)

vertices:
{'ID': 0, 'start': 660, 'end': 840, 'address_hash': b'\xb0\xb2\x10xT,\x10\x84\xe9\xa7\xf31\xe4\x12\xad\xb8"N\x19\xf1'} 
{'ID': 1, 'start': 540, 'end': 660, 'address_hash': b'\x08\xfa{s@-G\xa0N5\xbf:e\xfc(\x004\x82\xa7#'} 
{'ID': 2, 'start': 600, 'end': 720, 'address_hash': b'\xf4\xd9\xdd\xc6\xdaM~v\xd8\xfa\x83X4\xf4Q\x05aO\xc2\xa1'} 
{'ID': 3, 'start': 720, 'end': 840, 'address_hash': b'`\x80a\xb0N\xf5\xbe!\xc2\xcd\xd6\xab\x86\xa9z\xb9\xf2\xc2\x86Z'} 
{'ID': 4, 'start': 600, 'end': 678, 'address_hash': b'jC\xb6\xfc\xe8jgDQ\x96\x8a\xc5\x19s\xcdX\xcf\xda]\x0b'} 
{'ID': 5, 'start': 600, 'end': 720, 'address_hash': b'%(\x82\x85\xe9Z\xc3\x06\xed\x91Z\xe4\xc2\xf2\xf5\x9f\xe7\x98/h'} 
{'ID': 6, 'start': 600, 'end': 720, 'address_hash': b'\x7fn\xf1\x15\x03\x81\xd5\xc3\x06\x11\x133\xdd1#\x1f\xbe\x84y\xce'} 
{'ID': 7, 'start': 600, 'end': 720, 'address_hash': b'\xe2s\xe0\x1aA\xd7;\xd2-\x05\xb4}{\x8d\xaa\tK\xfe\x06t'} 
{'ID': 8, 'start': 660, 'end': 780, 'address_hash': b'\xaa\x84qy\x08

In [12]:
print(graph.get_edges_from_vid(10))

[[10, 0, 7.02], [10, 1, 38.06], [10, 2, 38.6], [10, 3, 39.26], [10, 4, 17.31], [10, 5, 13.58], [10, 6, 26.37], [10, 7, 31.42], [10, 8, 35.95], [10, 9, 30.48]]


In [21]:
'''
For the sake of initial testing, I am going to start by going to the first house on the list (ID = 0).
Also, the starting time will be that starting time of the vertex we're starting with.
    A better starting time can be defined later (i.e. when routing is solved).
'''
starting_id = 0
starting_vertex = graph.get_vertex_from_vid(starting_id)
start_time = starting_vertex['start']

In [30]:
def visit_next(graph, current_vertex, arrival_time, average_time_at_each_house = 30, visited = []):
    idx = current_vertex['ID']
    print('Arrived at vertex %d at time %d' % (idx, arrival_time))
    visited += [idx]
    arrival_time += average_time_at_each_house
    
    print('Leaving at vertex %d at time %d' % (idx, arrival_time))
    
    edges = graph.get_edges(current_vertex)
    edges_no_cycle = get_acyclic_edges(edges, visited)

    print(visited, edges_no_cycle)
    

def get_acyclic_edges(edges, visited):
    E = []
    for edge in edges:
        idx = edge[1]
        if been_visited(visited, idx):
            pass
        else:
            E += [edge]
    return E
        
def been_visited(visited, v):
    '''
    Checks to make sure you're not going back to a node that has already been visited.
    '''
    return v in visited

In [31]:
visit_next(graph, starting_vertex, start_time)

Arrived at vertex 0 at time 660
Leaving at vertex 0 at time 690
[0] [[0, 1, 40.45], [0, 2, 40.98], [0, 3, 36.66], [0, 4, 10.72], [0, 5, 19.29], [0, 6, 25.2], [0, 7, 25.56], [0, 8, 28.85], [0, 9, 30.41], [0, 10, 7.08]]
